In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import random
import warnings,logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [5]:
# !pip install --upgrade ai2thor --quiet
# !pip install ai2thor-colab prior --upgrade &> /dev/null

In [6]:
import os
os.system('apt-get install xvfb')

import prior
dataset = prior.load_dataset("procthor-10k")
house = dataset["train"][11]

# import ai2thor_colab
# ai2thor_colab.start_xserver()

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|███████████████████████| 1000/1000 [00:00<00:00, 7765.30it/s]


In [7]:
from ai2thor.controller import Controller
from ai2thor.platform import CloudRendering
from PIL import Image

controller = Controller(platform=CloudRendering)
# controller = Controller(scene='FloorPlan30',platform=CloudRendering)
# controller = Controller(scene=house, visibilityDistance=1.5 , width=150, height=100)
# img = Image.fromarray(controller.last_event.frame)

In [ ]:
# Function to teleport and get an image
def Teleporting():
    event = controller.step(action="GetReachablePositions")
    reachable_positions = event.metadata["actionReturn"]
    position = random.choice(reachable_positions)
    rotation = random.choice(range(360))
    event = controller.step(action="Teleport", position=position, rotation=rotation)
    img = Image.fromarray(controller.last_event.frame)
    return img, controller.last_event.frame

# Image to text

In [9]:
# ! pip install transformers -q

In [10]:
from transformers import pipeline

In [11]:
captioner = pipeline(model="ydshieh/vit-gpt2-coco-en")
# caption = pipeline('image-to-text')
def image_to_caption(img):
    caption = pipeline('image-to-text')
    image_text= caption(img)[0]
    return image_text

# Generate Images

In [12]:
# to generate data for evaluation
import time 
import pandas as pd
import os
import random
from PIL import Image
from transformers import pipeline


# Initialize a DataFrame to store image paths and captions
df = pd.DataFrame(columns=["Image Path", "Caption"])

# Directory to save images
image_dir = "images"

# Create the directory if it doesn't exist
os.makedirs(image_dir, exist_ok=True)

# Get the highest index in the image filenames
existing_image_indices = [int(filename.split(".")[0]) for filename in os.listdir(image_dir) if filename.endswith(".jpg")]
highest_index = max(existing_image_indices, default=-1)

# Load the existing CSV file if it exists
existing_csv_file_path = "image_captions_new.csv"
if os.path.exists(existing_csv_file_path):
    existing_df = pd.read_csv(existing_csv_file_path)
else:
    existing_df = pd.DataFrame(columns=["Image Path", "Caption"])

for i in range(highest_index + 1, highest_index + 1 + 150):
    img, x = Teleporting()
    
    # Generate a caption for the image
    caption_result = image_to_caption(img)
    caption = caption_result['generated_text']
    
    # Define the image filename
    image_filename = f"{i}.jpg"
    
    # Save the image to the specified directory
    img.save(os.path.join(image_dir, image_filename))
    
    # Add the image path and caption to the DataFrame
    df = pd.concat([df, pd.DataFrame({"Image Path": [os.path.join(image_dir, image_filename)], "Caption": [caption]})], ignore_index=True)
    
    time.sleep(1)

# Append the new data to the existing DataFrame
combined_df = pd.concat([existing_df, df], ignore_index=True)

# Save the updated DataFrame to the same CSV file
combined_df.to_csv(existing_csv_file_path, index=False, encoding='utf-8')
